In [4]:
# 丢弃法（Dropout）— 使用Gluon
# http://zh.gluon.ai/chapter_supervised-learning/dropout-gluon.html

from mxnet.gluon import nn

net = nn.Sequential()
drop_prob1 = 0.2
drop_prob2 = 0.5

with net.name_scope():
    net.add(nn.Flatten())
    # 第一层全连接。
    net.add(nn.Dense(256, activation='relu'))
#     net.add(nn.Dropout(drop_prob1))
    
    # 第二层全连接。
    net.add(nn.Dense(256, activation='relu', params=net[-1].params))
#     net.add(nn.Dropout(drop_prob2))
    
    net.add(nn.Dense(10))
net.initialize()

In [5]:
# 读取数据并训练
# 数据获取
import sys
sys.path.append('..')
import utils
from mxnet import gluon

batch_size = 256
def transform(data, label):
#     print(data.shape)
#     print(label.shape)
#     return nd.transpose(data.astype('float32'), (0,3,1,2))/255, label.astype('float32')
    return data.astype('float32') / 255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

/home/ly/.local/lib/python3.6/site-packages/mxnet/gluon/data/vision/datasets.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/home/ly/.local/lib/python3.6/site-packages/mxnet/gluon/data/vision/datasets.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


In [6]:
# 训练
from mxnet import autograd
from mxnet import gluon
from mxnet import nd

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd',{
    'learning_rate': 0.5
})

learning_rate = .5

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        print(net[1].weight.grad())
        print(net[2].weight.grad())
        break
        trainer.step(batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
    
    test_acc = utils.evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))

MXNetError: Shape inconsistent, Provided = [256,784], inferred shape=(256,256)